In [1]:
import pandas as pd
%matplotlib inline

In [2]:
xl = pd.ExcelFile('./data/superstore.xls') # Reads all the file with all sheets at once

In [4]:
xl.sheet_names

['Orders', 'Returns', 'People']

### 1. Parse/read the sheets separately

In [5]:
orders = xl.parse('Orders')

In [7]:
returns = xl.parse('Returns')
people = xl.parse('People')

### 2. Ways of joining

- `merge`: SQL-style of joining.
- `join`: join uses indices.
- `concat`: puts together rows and columns without checking for duplicates.

**Example** Joining `orders` with `returns`.

In [24]:
df = pd.merge(
    left=orders, 
    right=returns, 
    how='left', # inner, outer, left, right as in SQL
    left_on='Order ID', 
    right_on='Order ID')

In [25]:
orders.shape

(9994, 21)

In [26]:
returns.shape

(296, 2)

In [27]:
df.shape

(9994, 22)

### Using indices

If we set up an index, we can speed up the join operation.

In [28]:
returns_with_idx = returns.set_index('Order ID') # Define the index and copy into new df

In [32]:
df_with_idx = pd.merge(
    left=orders, 
    right=returns_with_idx, # indexed version of the returns dataframe
    how='left', 
    left_on='Order ID', 
    right_index=True)

In [37]:
%timeit pd.merge(left=orders, right=returns, how='left', left_on='Order ID', right_on='Order ID')

7.91 ms ± 186 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [38]:
%timeit pd.merge(left=orders, right=returns_with_idx, how='left', left_on='Order ID', right_index=True)

3.6 ms ± 117 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


**Remark:** Good idea to use indices :)

## Using `join`

In [42]:
df_with_join = orders.join(returns_with_idx, on='Order ID', how='left') # Same as pd.merge() with index

In [43]:
df_with_join.shape

(9994, 22)

## Using `concat`

In [45]:
row_stack = pd.concat([orders, returns], axis=0) # stacks columns/rows 

C:\Users\HP\Anaconda3\envs\a1\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [46]:
row_stack.shape

(10290, 22)

In [47]:
orders.shape

(9994, 21)

In [48]:
returns.shape

(296, 2)

In [50]:
row_stack['Returned'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: Returned, dtype: object

In [51]:
col_stack = pd.concat([orders, returns], axis=1)

In [53]:
col_stack.shape

(9994, 23)

In [52]:
col_stack.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Returned,Order ID
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,Yes,CA-2017-153822
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,Yes,CA-2017-129707
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,Yes,CA-2014-152345
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,Yes,CA-2015-156440
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,Yes,US-2017-155999


## Concat and drop duplicates

In [55]:
orders2 = orders.head()

In [66]:
dup_orders = pd.concat([orders, orders2], axis=0)

In [67]:
dup_orders.shape

(9999, 21)

In [68]:
dup_orders[dup_orders['Row ID']==1]

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136


In [61]:
dedup_orders = dup_orders.drop_duplicates()

In [63]:
dedup_orders.shape

(9994, 21)

In [64]:
dup_orders.drop_duplicates(inplace=True, subset=['Order ID','Row ID'], keep='first')

# Your turn!

- Find the sales person who should get a bonus. This means, the person who has the highest profit on non-returned orders.

    - **Strategy:** 
        - Join `orders` and `returns`. 
        - Keep those that are not returned (`Returned`!=`Yes`).
        - Group by and sum.
        - Finally, join with `people`.
    

In [71]:
orders_returns = pd.merge(left=orders, 
                          right=returns, 
                          how='left', 
                          left_on='Order ID', 
                          right_on='Order ID')

In [73]:
orders_returns.shape

(9994, 22)

In [74]:
orders_returns_not_returned = orders_returns[orders_returns['Returned'] != 'Yes']

In [76]:
almost_done = orders_returns_not_returned[['Profit','Region']]

In [77]:
final = pd.merge(left=almost_done, right=people, left_on='Region', right_on='Region', how='left')

In [78]:
final.head()

,Profit,Region,Person
0,41.9136,South,Cassandra Brandow
1,219.5820,South,Cassandra Brandow
2,6.8714,West,Anna Andreadi
3,-383.0310,South,Cassandra Brandow
4,2.5164,South,Cassandra Brandow


In [81]:
final.groupby('Person')['Profit'].sum()

Person
Anna Andreadi        88755.0711
Cassandra Brandow    44530.8198
Chuck Magee          86537.9794
Kelly Williams       43340.7899
Name: Profit, dtype: float64

In [85]:
final.groupby('Person')['Profit'].sum().sort_values(ascending=False)

Person
Anna Andreadi        88755.0711
Chuck Magee          86537.9794
Cassandra Brandow    44530.8198
Kelly Williams       43340.7899
Name: Profit, dtype: float64